In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import orjson as json

from caveclient import CAVEclient
from standard_transform.datasets import minnie_ds, v1dd_ds

v1dd_feat_dir = Path('features/v1dd_spine')
v1dd_file = 'features/v1dd_spine/feat_df_spine_axon_prediction.feather'
v1dd_df = pd.read_feather(v1dd_file)

In [ ]:
v1dd_rids = v1dd_df['root_id'].values

In [ ]:
import tqdm.autonotebook as tqdm

In [ ]:
v1dd_syn_depths = []
for rid in tqdm.tqdm(v1dd_rids):
    with open(v1dd_feat_dir / f"{rid}.json", 'r') as f:
        syn_data = json.loads(f.read())
        if syn_data.get('num_syn_dendrite', 0) < 500:
            continue
        if syn_data.get('syn_depth_dist_spine') is not None:
            v1dd_syn_depths.append(syn_data['syn_depth_dist_spine'])

In [ ]:
v1dd_syn_depths = np.concat(v1dd_syn_depths)

In [ ]:
v1dd_syn_df = pd.DataFrame(v1dd_syn_depths, columns=['syn_depth'])
v1dd_syn_df['dataset'] = 'v1dd'

In [ ]:
microns_feat_dir = Path('features/minnie65_phase3_v1_spines')
microns_file = 'features/minnie65_phase3_v1_spines/feat_df_spine_axon_prediction.feather'
microns_df = pd.read_feather(microns_file)

In [ ]:
microns_rids= microns_df['root_id'].values

In [ ]:
microns_syn_depths = []
for rid in tqdm.tqdm(microns_rids):
    with open(microns_feat_dir / f"{rid}.json", 'r') as f:
        syn_data = json.loads(f.read())
        if syn_data.get('syn_depth_dist_spine') is not None:
            microns_syn_depths.append(syn_data['syn_depth_dist_spine'])

microns_syn_depths = np.concatenate(microns_syn_depths)
microns_syn_df = pd.DataFrame(microns_syn_depths, columns=['syn_depth'])
microns_syn_df['dataset'] = 'microns'

## Compare depth distributions

In [ ]:
combo_df = pd.concat([v1dd_syn_df, microns_syn_df], ignore_index=True)
v1dd_idx = combo_df.query('dataset == "v1dd"').index
combo_df.loc[v1dd_idx, 'syn_depth'] = 15 + 0.93 * combo_df['syn_depth'].loc[v1dd_idx]

In [ ]:
fig, ax = plt.subplots(figsize=(4, 8))

sns.histplot(
    y='syn_depth',
    data=combo_df.sample(1_000_000),
    hue='dataset',
    stat='count',
    common_norm=False,
    bins=100,
    element='step',
)

ax.invert_yaxis()

Use soma depth instead

In [ ]:
soma_depth_df = pd.DataFrame(
    {
        'soma_depth': np.concat([microns_df['soma_depth'].values, v1dd_df['soma_depth'].values]),
        'dataset': ['microns'] * len(microns_df) + ['v1dd'] * len(v1dd_df)
    }
)

In [ ]:
v1dd_neuron_df = pd.read_feather(
    'data/v1dd_nucleus_soma_model_results.feather'
)

In [ ]:
v1dd_neuron_df = v1dd_neuron_df.query('coarse_label_pred == 0').reset_index(drop=True)

In [ ]:
soma_depth_df = pd.DataFrame(
    {
        'soma_depth': np.concat([microns_df['soma_depth'].values, 15+0.94*v1dd_df['soma_depth'].values, 15+0.94*v1dd_neuron_df['soma_depth'].values]),
        'dataset': ['microns'] * len(microns_df) + ['v1dd_ss'] * len(v1dd_df) + ['v1dd_neuron'] * len(v1dd_neuron_df)
    }
)
fig, ax = plt.subplots(figsize=(8, 8))

sns.histplot(
    y='soma_depth',
    data=soma_depth_df,
    hue='dataset',
    stat='count',
    palette={'microns': 'black', 'v1dd_ss': 'darkgreen', 'v1dd_neuron': 'darkmagenta'},
    common_norm=False,
    bins=75,
    element='step',
    alpha=1,
    fill=False,
)

ax.invert_yaxis()

### Apples to apples using streamslines and the same radial distance

In [ ]:
v1dd_center = np.array([95473, 50500, 5077]) * [9,9,45]
microns_center = np.array([180112, 114928, 21520]) * [4, 4, 40]

In [ ]:
client_minnie = CAVEclient('minnie65_phase3_v1')
client_v1dd = CAVEclient('v1dd')

In [ ]:
tbl_qry_minnie = client_minnie.materialize.tables
tbl_qry_v1dd = client_v1dd.materialize.tables

In [ ]:
minnie_radial_df = tbl_qry_minnie.aibs_metamodel_mtypes_v661_v2.get_all(desired_resolution=[1,1,1])
v1dd_radial_df = tbl_qry_v1dd.nucleus_detection_v0.get_all(desired_resolution=[1,1,1])

In [ ]:
minnie_radial_df['rad_dist'] = minnie_ds.streamline_nm.radial_distance(microns_center, np.vstack(minnie_radial_df['pt_position'].values))

In [ ]:
v1dd_radial_df.head()

In [ ]:
v1dd_radial_df = v1dd_radial_df.query('id in @v1dd_neuron_df.cell_id ').reset_index(drop=True)

In [ ]:
v1dd_radial_df['rad_dist'] = v1dd_ds.streamline_nm.radial_distance(
    v1dd_center,
    np.vstack(v1dd_radial_df['pt_position'].values),
)

In [ ]:
minnie_sample = minnie_radial_df.query('rad_dist < 150')

In [ ]:
v1dd_sample = v1dd_radial_df.query('rad_dist < 150')

In [ ]:
minnie_sample['soma_depth'] = minnie_ds.transform_nm.apply_dataframe('pt_position', minnie_sample, projection='y')
v1dd_sample['soma_depth'] = v1dd_ds.transform_nm.apply_dataframe('pt_position', v1dd_sample, projection='y')

In [ ]:
soma_depth_sample_df = pd.DataFrame(
    {
        'soma_depth': np.concat([minnie_sample['soma_depth'].values, -10+0.98*v1dd_sample['soma_depth'].values]),
        'dataset': ['microns'] * len(minnie_sample) + ['v1dd'] * len(v1dd_sample)
    }
)
fig, ax = plt.subplots(figsize=(8, 8))

sns.histplot(
    y='soma_depth',
    data=soma_depth_sample_df,
    hue='dataset',
    stat='count',
    palette={'microns': 'green', 'v1dd': 'darkmagenta'},
    common_norm=False,
    bins=50,
    element='step',
    alpha=0.1,
    # fill=False,
)

ax.invert_yaxis()

General lesson is that v1dd is largely aligned in depth, with maybe a 10-15 µm offset in soma depth.

There remains a big difference `in the distribution of layer 2–4 cells and layer 6 cells. For some reason, the overall count of layer 6 is basically the same across teh two datasets, while layer 2-4 cells are considerably less numerous. by a factor of 20% or so.  